<a href="https://colab.research.google.com/github/joe-granick/CS462/blob/main/GNN_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip list | grep torch

torch                              2.5.1+cu121
torchaudio                         2.5.1+cu121
torchsummary                       1.5.1
torchvision                        0.20.1+cu121


In [173]:
!ls ml-1m


ml-1m


In [189]:
!wget https://files.grouplens.org/datasets/movielens/ml-1m.zip
!unzip ml-1m.zip -d ml-1m


--2024-11-18 04:11:06--  https://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  3.06MB/s    in 1.8s    

2024-11-18 04:11:09 (3.06 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]

Archive:  ml-1m.zip
   creating: ml-1m/ml-1m/
  inflating: ml-1m/ml-1m/movies.dat  
  inflating: ml-1m/ml-1m/ratings.dat  
  inflating: ml-1m/ml-1m/README      
  inflating: ml-1m/ml-1m/users.dat   


In [197]:
!cd ../
!ls


bin			    dev     lib32   ml-1m.zip		      python-apt	 srv	var
boot			    etc     lib64   mnt			      python-apt.tar.xz  sys
content			    home    libx32  NGC-DL-CONTAINER-LICENSE  root		 tmp
cuda-keyring_1.0-1_all.deb  kaggle  media   opt			      run		 tools
datalab			    lib     ml-1m   proc		      sbin		 usr


In [199]:
cd ml-1m

/ml-1m


In [201]:
ls ml-1m

movies.dat  ratings.dat  README  users.dat


In [203]:
import pandas as pd

# Load the datasets with specified encoding
ratings = pd.read_csv('ml-1m/ratings.dat', sep='::', header=None, engine='python',
                      names=['UserID', 'MovieID', 'Rating', 'Timestamp'], encoding='latin1')
movies = pd.read_csv('ml-1m/movies.dat', sep='::', header=None, engine='python',
                     names=['MovieID', 'Title', 'Genres'], encoding='latin1')
users = pd.read_csv('ml-1m/users.dat', sep='::', header=None, engine='python',
                    names=['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'], encoding='latin1')

# Display the first few rows to confirm successful loading
print("Ratings Data:")
print(ratings.head())
print("\nMovies Data:")
print(movies.head())
print("\nUsers Data:")
print(users.head())


Ratings Data:
   UserID  MovieID  Rating  Timestamp
0       1     1193       5  978300760
1       1      661       3  978302109
2       1      914       3  978301968
3       1     3408       4  978300275
4       1     2355       5  978824291

Movies Data:
   MovieID                               Title                        Genres
0        1                    Toy Story (1995)   Animation|Children's|Comedy
1        2                      Jumanji (1995)  Adventure|Children's|Fantasy
2        3             Grumpier Old Men (1995)                Comedy|Romance
3        4            Waiting to Exhale (1995)                  Comedy|Drama
4        5  Father of the Bride Part II (1995)                        Comedy

Users Data:
   UserID Gender  Age  Occupation Zip-code
0       1      F    1          10    48067
1       2      M   56          16    70072
2       3      M   25          15    55117
3       4      M   45           7    02460
4       5      M   25          20    55455


In [204]:
# Ratings
print("Number of Ratings:", ratings.shape[0])
print("Number of Users:", ratings['UserID'].nunique())
print("Number of Movies:", ratings['MovieID'].nunique())

# Movies
print("Number of Movies:", movies.shape[0])

# Users
print("Number of Users:", users.shape[0])
print("Unique Occupations:", users['Occupation'].nunique())


Number of Ratings: 1000209
Number of Users: 6040
Number of Movies: 3706
Number of Movies: 3883
Number of Users: 6040
Unique Occupations: 21


In [205]:
import torch
from torch_geometric.data import Data

# Create edge index: Users -> Movies
edge_index = torch.tensor(ratings[['UserID', 'MovieID']].values.T - 1, dtype=torch.long)  # Adjust to 0-based index

# Create edge attributes (ratings)
edge_attr = torch.tensor(ratings['Rating'].values, dtype=torch.float).view(-1, 1)

# Create node features
num_users = users['UserID'].max()
num_movies = movies['MovieID'].max()
num_nodes = num_users + num_movies

# One-hot encoding for user/movie nodes
node_features = torch.eye(num_nodes, dtype=torch.float)

# GNN Data Object
data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_attr)
print(data)


Data(x=[9992, 9992], edge_index=[2, 1000209], edge_attr=[1000209, 1])


In [1]:
!pip install torch==2.0.1+cu117 torchvision==0.15.2+cu117 torchaudio==2.0.2+cu117 --index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://download.pytorch.org/whl/cu117


In [3]:
!pip install torch-scatter==2.1.2+pt20cu117 \
    torch-sparse==0.6.18+pt20cu117 \
    torch-cluster==1.6.3+pt20cu117 \
    torch-spline-conv==1.2.2+pt20cu117 \
    -f https://data.pyg.org/whl/torch-2.0.0+cu117.html

Looking in links: https://data.pyg.org/whl/torch-2.0.0+cu117.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 106.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 100.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.2/887.2 kB 55.9 MB/s eta 0:00:00


In [2]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.6 MB/s eta 0:00:00


In [6]:
!wget https://files.grouplens.org/datasets/movielens/ml-1m.zip
!unzip ml-1m.zip -d ml-1m


--2024-11-18 16:04:06--  https://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  25.7MB/s    in 0.2s    

2024-11-18 16:04:06 (25.7 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]

Archive:  ml-1m.zip
   creating: ml-1m/ml-1m/
  inflating: ml-1m/ml-1m/movies.dat  
  inflating: ml-1m/ml-1m/ratings.dat  
  inflating: ml-1m/ml-1m/README      
  inflating: ml-1m/ml-1m/users.dat   


In [9]:
!ls ml-1m/ml-1m

movies.dat  ratings.dat  README  users.dat


In [15]:
import torch
from torch_geometric.data import Data
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# Load and preprocess your dataset
ratings = pd.read_csv('ml-1m/ml-1m/ratings.dat', sep='::', header=None, engine='python',
                      names=['UserID', 'MovieID', 'Rating', 'Timestamp'], encoding='latin1')
movies = pd.read_csv('ml-1m/ml-1m/movies.dat', sep='::', header=None, engine='python',
                     names=['MovieID', 'Title', 'Genres'], encoding='latin1')
users = pd.read_csv('ml-1m/ml-1m/users.dat', sep='::', header=None, engine='python',
                    names=['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'], encoding='latin1')

# Reindex UserID and MovieID
user_id_map = {id: idx for idx, id in enumerate(users['UserID'].unique())}
movie_id_map = {id: idx for idx, id in enumerate(movies['MovieID'].unique())}

ratings['UserID'] = ratings['UserID'].map(user_id_map)
ratings['MovieID'] = ratings['MovieID'].map(movie_id_map) + len(user_id_map)  # Offset movies

num_users = len(user_id_map)
num_movies = len(movie_id_map)
total_nodes = num_users + num_movies

# Create features and edge attributes
node_features = torch.randn(total_nodes, 7)  # Example: 7-dimensional node features
edge_index = torch.tensor(ratings[['UserID', 'MovieID']].values.T, dtype=torch.long)

edge_attr = torch.tensor(ratings['Rating'].values, dtype=torch.float32).unsqueeze(1)
target = (ratings['Rating'] >= 4).astype(int).values  # Binary target for rating >= 4

# Split dataset into train and validation
train_idx, val_idx = train_test_split(np.arange(edge_index.shape[1]), test_size=0.2, random_state=42)
train_data = Data(x=node_features, edge_index=edge_index[:, train_idx], edge_attr=edge_attr[train_idx],
                  y=torch.tensor(target[train_idx], dtype=torch.float32))
val_data = Data(x=node_features, edge_index=edge_index[:, val_idx], edge_attr=edge_attr[val_idx],
                y=torch.tensor(target[val_idx], dtype=torch.float32))

# Validate Data
print("Node Feature Matrix Shape:", node_features.shape)
print("Edge Index Shape:", edge_index.shape)
print("Max Edge Index:", edge_index.max().item())


Node Feature Matrix Shape: torch.Size([9923, 7])
Edge Index Shape: torch.Size([2, 1000209])
Max Edge Index: 9922


In [16]:
print("Edge Attribute Shape:", edge_attr.shape)


Edge Attribute Shape: torch.Size([1000209, 1])


In [12]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GNNEdgeClassifier(nn.Module):
    def __init__(self, hidden_dim=16, edge_feature_dim=5):
        super(GNNEdgeClassifier, self).__init__()
        self.gcn1 = GCNConv(-1, hidden_dim)  # Input dimension inferred dynamically
        self.gcn2 = GCNConv(hidden_dim, hidden_dim)
        self.edge_predictor = nn.Sequential(
            nn.LazyLinear(hidden_dim),  # Infer input size lazily
            nn.ReLU(),
            nn.LazyLinear(1)  # Infer input size lazily for final output
        )

    def forward(self, x, edge_index, edge_attr):
        # GCN Layers
        x = F.relu(self.gcn1(x, edge_index))
        x = F.relu(self.gcn2(x, edge_index))

        # Aggregate edge features
        edge_src = x[edge_index[0]]  # Source node features
        edge_tgt = x[edge_index[1]]  # Target node features
        edge_features = torch.cat([edge_src, edge_tgt, edge_attr], dim=1)

        # Edge prediction
        return self.edge_predictor(edge_features).squeeze(-1)


In [17]:
# GAT-based Model
class GATEdgeClassifier(nn.Module):
    def __init__(self, hidden_dim=16):
        super(GATEdgeClassifier, self).__init__()
        self.gat1 = GATConv(-1, hidden_dim, heads=4, concat=True)
        self.gat2 = GATConv(hidden_dim * 4, hidden_dim, heads=1, concat=False)
        self.edge_predictor = nn.Sequential(
            nn.Linear(hidden_dim * 2 + 1, hidden_dim),  # Includes edge_attr
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, x, edge_index, edge_attr):
        x = F.relu(self.gat1(x, edge_index))
        x = F.relu(self.gat2(x, edge_index))
        edge_src = x[edge_index[0]]
        edge_tgt = x[edge_index[1]]
        edge_features = torch.cat([edge_src, edge_tgt, edge_attr], dim=1)
        return self.edge_predictor(edge_features).squeeze(-1)


In [19]:
import torch.optim as optim
import torch.nn.init as init

# Initialize the model
model = GATEdgeClassifier(hidden_dim=16)

# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# (Optional) Define a custom weight initialization function
def initialize_weights(m):
    if isinstance(m, nn.Linear):  # Applies to the edge predictor
        init.xavier_uniform_(m.weight)
        if m.bias is not None:
            init.zeros_(m.bias)

# Apply custom initialization
model.apply(initialize_weights)

# Print the model to ensure it looks correct
print(model)



GATEdgeClassifier(
  (gat1): GATConv(-1, 16, heads=4)
  (gat2): GATConv(64, 16, heads=1)
  (edge_predictor): Sequential(
    (0): Linear(in_features=33, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=1, bias=True)
  )
)


In [20]:
# Example training loop
for epoch in range(50):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    train_output = model(train_data.x, train_data.edge_index, edge_attr=train_data.edge_attr).squeeze(-1)

    # Compute loss
    train_loss = criterion(train_output, train_data.y.float())

    # Backpropagation
    train_loss.backward()
    optimizer.step()

    # Evaluation
    model.eval()
    with torch.no_grad():
        val_output = model(val_data.x, val_data.edge_index, edge_attr=val_data.edge_attr).squeeze(-1)
        val_predictions = (torch.sigmoid(val_output) > 0.5).long()
        val_accuracy = (val_predictions == val_data.y.long()).sum().item() / val_data.y.size(0)

    print(f"Epoch {epoch+1}, Train Loss: {train_loss.item():.4f}, Val Accuracy: {val_accuracy:.4f}")


Epoch 1, Train Loss: 0.6402, Val Accuracy: 0.5762
Epoch 2, Train Loss: 0.6078, Val Accuracy: 0.5915
Epoch 3, Train Loss: 0.5991, Val Accuracy: 0.6314
Epoch 4, Train Loss: 0.5893, Val Accuracy: 0.6832
Epoch 5, Train Loss: 0.5785, Val Accuracy: 0.7406
Epoch 6, Train Loss: 0.5656, Val Accuracy: 0.7821
Epoch 7, Train Loss: 0.5534, Val Accuracy: 0.8023
Epoch 8, Train Loss: 0.5406, Val Accuracy: 0.7937
Epoch 9, Train Loss: 0.5261, Val Accuracy: 0.7763
Epoch 10, Train Loss: 0.5128, Val Accuracy: 0.7813
Epoch 11, Train Loss: 0.4985, Val Accuracy: 0.8157
Epoch 12, Train Loss: 0.4818, Val Accuracy: 0.8532
Epoch 13, Train Loss: 0.4673, Val Accuracy: 0.8388
Epoch 14, Train Loss: 0.4509, Val Accuracy: 0.8115
Epoch 15, Train Loss: 0.4364, Val Accuracy: 0.8146
Epoch 16, Train Loss: 0.4227, Val Accuracy: 0.8411
Epoch 17, Train Loss: 0.4096, Val Accuracy: 0.8569
Epoch 18, Train Loss: 0.3967, Val Accuracy: 0.8469
Epoch 19, Train Loss: 0.3822, Val Accuracy: 0.8489
Epoch 20, Train Loss: 0.3683, Val Accura